### Manipulation of Peaks de novo results of Prochlorococus MED4 LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)

For technical duplicates, I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [34]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/PeaksDN/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/PeaksDN


In [35]:
ls

RAL95_MED2_trypsin_1_PTMopt_DN50.csv


In [36]:
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [37]:
#read the CSV into a datadrame we name 'comet' using the pandas read_csv function
peaks = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/PeaksDN/RAL95_MED2_trypsin_1_PTMopt_DN50.csv")


#look at the dataframe
peaks.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,3,15768,022016_RAL4_95_MED2_trypsin_1.raw,KELN(+.98)LDTDLGK,11,98,11,623.8295,2,26.80,2620000.0,1245.6453,-0.6,Deamidation (NQ),98 100 99 99 99 99 99 99 100 99 99,KELN(+.98)LDTDLGK,CID
1,3,6171,022016_RAL4_95_MED2_trypsin_1.raw,KDLESLDSTNK,11,98,11,625.3166,2,12.87,4180000.0,1248.6196,-0.8,NaN,98 100 100 100 99 99 99 97 98 99 99,KDLESLDSTNK,CID
2,3,46585,022016_RAL4_95_MED2_trypsin_1.raw,FFLLFK,6,98,6,407.7466,2,68.38,7850000.0,813.4788,-0.3,NaN,96 98 99 100 99 99,FFLLFK,CID
3,3,28681,022016_RAL4_95_MED2_trypsin_1.raw,KLFTDYQELMK,11,98,11,708.3657,2,44.32,4690000.0,1414.7166,0.2,NaN,99 100 99 99 99 96 94 99 99 99 99,KLFTDYQELMK,CID
4,3,39806,022016_RAL4_95_MED2_trypsin_1.raw,WALEELLNK,9,98,9,558.3083,2,59.10,43600000.0,1114.6023,-0.2,NaN,98 99 100 100 99 97 98 97 98,WALEELLNK,CID


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ
    variable iron cation on lysine: 54.010565 K
    variable methylation of lysine and arginine: 14.015650 KR

We'll then write this manipulated dataframe to a new file.

In [38]:
# use a count function to enumerate the # of ccarbamidomethylated C's in each peptide
peaks['c-carb'] = peaks['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks['m-oxid'] = peaks['Peptide'].str.count("15.99")

# use a count function to enumerate the # of deamidated N's in each peptide
peaks['n-deam'] = peaks['Peptide'].str.count("N\(+.98")

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks['q-deam'] = peaks['Peptide'].str.count("Q\(+.98")

# use a count function to enumerate the # of iron adducted K's in each peptide
peaks['k-iron'] = peaks['Peptide'].str.count("53.92")

# use a count function to enumerate the # of methylated K's in each peptide
peaks['k-meth'] = peaks['Peptide'].str.count("K\(+14.02")

# use a count function to enumerate the # of methylated R's in each peptide
peaks['r-meth'] = peaks['Peptide'].str.count("R\(+14.02")

# create a column with 'stripped' peptide sequences using strip
peaks['stripped peptide'] = peaks['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks['stripped length'] = peaks['stripped peptide'].apply(len)

# write modified dataframe to new txt file, same name + 'stripped'
peaks.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/PeaksDN/RAL95_MED2_trypsin_1_PTMopt_DN50_stripped.csv")


# check out the results
peaks.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,mode,c-carb,m-oxid,n-deam,q-deam,k-iron,k-meth,r-meth,stripped peptide,stripped length
0,3,15768,022016_RAL4_95_MED2_trypsin_1.raw,KELN(+.98)LDTDLGK,11,98,11,623.8295,2,26.80,...,CID,0,0,0,0,0,0,0,KELNLDTDLGK,11
1,3,6171,022016_RAL4_95_MED2_trypsin_1.raw,KDLESLDSTNK,11,98,11,625.3166,2,12.87,...,CID,0,0,0,0,0,0,0,KDLESLDSTNK,11
2,3,46585,022016_RAL4_95_MED2_trypsin_1.raw,FFLLFK,6,98,6,407.7466,2,68.38,...,CID,0,0,0,0,0,0,0,FFLLFK,6
3,3,28681,022016_RAL4_95_MED2_trypsin_1.raw,KLFTDYQELMK,11,98,11,708.3657,2,44.32,...,CID,0,0,0,0,0,0,0,KLFTDYQELMK,11
4,3,39806,022016_RAL4_95_MED2_trypsin_1.raw,WALEELLNK,9,98,9,558.3083,2,59.10,...,CID,0,0,0,0,0,0,0,WALEELLNK,9
